## Setup

In [ ]:
!pip install poetry

In [ ]:
!pip install https://github.com/swanandlab/SMorph/releases/download/v0.1.1-alpha/SMorph-0.1.1.tar.gz

In [ ]:
on_colab = 'google.colab' in str(get_ipython())

import warnings
warnings.filterwarnings('ignore')
if not on_colab:
    import napari
import smorph.util.autocrop as ac

In [ ]:
# Helper function
def view_3D(*args):
    """Views a 3D image."""
    n_images = len(args)
    if not on_colab and n_images > 0:  
        with napari.gui_qt():
            viewer = napari.view_image(**args[0], ndisplay=3)
            for itr in range(1, n_images):
                if args[itr]['data'].max() > 1 and args[itr]['data'].max() % 1 == 0:
                    if 'colormap' in args[itr].keys():
                        del args[itr]['colormap']
                    if 'gamma' in args[itr].keys():
                        del args[itr]['gamma']
                    viewer.add_labels(**args[itr])
                else:
                    viewer.add_image(**args[itr])

In [ ]:
SOMA_SELECTED = 'Autocropped/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 1_LEFT HILUS_21 DAY-HilusConEQ/residue'

## Select somas of individual cells in clumps

### Option 1. Pre-selected point ROIs

In [ ]:
reconstructed_labels = None
reconstructed_labels, parent_path, roi_path = ac.postprocess_segment(SOMA_SELECTED, reconstructed_labels)

In [ ]:
view_3D({'data': reconstructed_labels, 'colormap': 'inferno', 'name': 'reconstructed_labels'})

### Option 2. Manually-select point ROIs

In [ ]:
reconstructed_seg = reconstructed_labels = None
reconstructed_seg, parent_path, roi_path, somas_est = ac.manual_postprocess(SOMA_SELECTED, reconstructed_seg)

In [ ]:
if not on_colab:
    with napari.gui_qt():
        man_viewer = napari.view_image(reconstructed_seg)
        man_viewer.add_points(somas_est, face_color='red', size=5, symbol='cross')

In [ ]:
import json
from os import listdir

import numpy as np
import tifffile
from skimage.measure import label

somas_est = man_viewer.layers[1].data

for file in listdir(SOMA_SELECTED):
    if not file.startswith('.') and file.endswith(('.tif', '.tiff')) and '_mip' not in file:
        name = SOMA_SELECTED + '/' + file
        image = tifffile.TiffFile(name)
        metadata = image.pages[0].tags['ImageDescription'].value
        metadata = json.loads(metadata)

        try:
            im = image.asarray()

            if reconstructed_labels is None:
                parent = ac._io.import_confocal_image(parent_path)
                reconstructed_labels = np.zeros(parent.shape)

            minz, miny, minx, maxz, maxy, maxx = metadata['bounds']
            linebuilder = ac._roi_extract._load_ROI(roi_path)
            X, Y = ac.core._unwrap_polygon(linebuilder)
            min_x, max_x = int(min(X)), int(max(X) + 1)
            min_y, max_y = int(min(Y)), int(max(Y) + 1)
            miny += min_y
            maxy += min_y
            minx += min_x
            maxx += min_x
            ll = np.array([minz, miny, minx])  # lower-left
            ur = np.array([maxz, maxy, maxx])  # upper-right
            inidx = np.all(np.logical_and(ll <= somas_est, somas_est <= ur), axis=1)
            somas_coords = somas_est[inidx]
            somas_coords -= np.array([minz+1, miny+1, minx+1])

            markers = np.zeros(im.shape)
            for i in range(1, somas_coords.shape[0] + 1):
                markers[tuple(map(int, somas_coords[i-1]))] = i

            labels = ac._postprocessing._segment_clump(im, markers)
            reconstructed_labels[minz:maxz, miny:maxy, minx:maxx] += labels
        except Exception as e:
            print(e)
reconstructed_labels = label(reconstructed_labels)

In [ ]:
view_3D({'data': reconstructed_seg, 'colormap': 'inferno', 'name': 'reconstructed_seg'},
        {'data': reconstructed_labels, 'colormap': 'inferno', 'name': 'reconstructed_labels'})

In [ ]:
reconstructed_volume = (reconstructed_labels > 0).sum()
f'Postreconstructing Volume: {reconstructed_volume}'

In [ ]:
linebuilder = ac._roi_extract._load_ROI(roi_path)

In [ ]:
import numpy as np
from skimage.morphology import binary_erosion
from skimage.segmentation import clear_border
from skimage.draw import polygon2mask

conservative = True
reconstructed_residue_labels = clear_border(reconstructed_labels, mask=None if conservative
                        else ac.core._compute_convex_hull(thresholded))

if linebuilder is not None:
    X, Y = ac.core._unwrap_polygon(linebuilder)
    min_x, max_x = int(min(X)), int(max(X) + 1)
    min_y, max_y = int(min(Y)), int(max(Y) + 1)
    shape = reconstructed_labels.shape
    roi_mask = np.empty(shape)
    roi_mask[0] = polygon2mask(shape[1:][::-1], list(zip(X, Y))).T

    for i in range(1, shape[0]):
        roi_mask[i] = roi_mask[0]
    roi_mask = binary_erosion(roi_mask)
    reconstructed_residue_labels = clear_border(reconstructed_residue_labels, mask=roi_mask)

reconstructed_filtered_regions = ac.arrange_regions(reconstructed_residue_labels)

In [ ]:
view_3D({'data': reconstructed_residue_labels, 'colormap': 'gray', 'name': 'reconstructed_filtered_regions'},
        {'data': reconstructed_labels, 'gamma': .8, 'colormap': 'inferno', 'name': 'reconstructed_labels'},
        {'data': roi_mask, 'gamma': .8, 'colormap': 'inferno', 'name': 'roi_mask'})

In [ ]:
X, Y = ac.core._unwrap_polygon(linebuilder)
min_x, max_x = int(min(X)), int(max(X) + 1)
min_y, max_y = int(min(Y)), int(max(Y) + 1)

In [ ]:
reconstructed_residue_labels = ac.filter_labels(reconstructed_labels[:, min_y:max_y, min_x:max_x], None, linebuilder, True)

In [ ]:
reconstructed_residue_volume = (reconstructed_residue_labels > 0).sum()
f'Postreconstructing-residue Volume: {reconstructed_residue_volume}; ' \
f'%age volume: {(reconstructed_residue_volume) / reconstructed_volume * 100} %'

In [ ]:
view_3D(#{'data': denoised, 'colormap': 'inferno', 'name': 'denoised'},
        {'data': reconstructed_residue_labels, 'colormap': 'gray', 'gamma': .8, 'name': 'reconstructed_filtered_labels'})
        #{'data': filtered_labels, 'colormap': 'inferno', 'gamma': .8, 'name': 'filtered_labels'},
        #{'data': labels, 'colormap': 'gist_earth', 'gamma': .8, 'name': 'labels'})

In [ ]:
reconstructed_filtered_regions = ac.arrange_regions(reconstructed_residue_labels)

LOW_VOLUME_CUTOFF = 0  # filter noise/artifacts
HIGH_VOLUME_CUTOFF = 1e9  # filter cell clusters

OUTPUT_OPTION = 'both'  # '3d' for 3D cells, 'mip' for Max Intensity Projections
SEGMENT_TYPE = 'segmented'

ac.export_cells(parent_path, LOW_VOLUME_CUTOFF,
                HIGH_VOLUME_CUTOFF, OUTPUT_OPTION, denoised,
                reconstructed_filtered_regions, None, SEGMENT_TYPE, NAME_ROI, linebuilder, roi_path=roi_path)